In [ ]:
import tarfile
import re
from datetime import datetime
from datetime import date
import math
import seaborn as sns
import pandas as pd
import json
import numpy as np
sns.set()
# Apply the default theme
sns.set_theme()
def remap_keys(mapping):
    return [{'key':k, 'value': v} for k, v in mapping.items()]

In [ ]:
tar = tarfile.open("/content/drive/MyDrive/enron_mail_20150507.tar.gz", "r:gz")
members = tar.getmembers()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BCC_CONTENT = True
CC_CONTENT = True

In [ ]:
content_mail = ""
email_id = {}
id = 0
data_simplified = []
allowed_keys = ["Date","From","To",'X-From','X-To']

if BCC_CONTENT:
  allowed_keys.append('Bcc')
  allowed_keys.append('X-bcc')

if CC_CONTENT:
  allowed_keys.append('Cc')
  allowed_keys.append('X-cc')


set_name_dir = ['_sent','_sent_mail','sent','sent_items']
for member in members:
     if len(member.name.split('/'))>1 and len([x for x in member.name.split('/') if x in set_name_dir]) and str(member.name) == "maildir/bass-e/sent/9.": 
        f = tar.extractfile(member)
        if f is not None:
          content_mail = str(f.read())
          name_person = member.name.split('/')[1]
          content_mail = content_mail.split('X-FileName:')[0]
          splited_content_mail = content_mail.split("\\n")
          dict_instance = {}
          for idx,instance in enumerate(splited_content_mail):
            splited_instance = instance.split(':')
            if splited_instance[0] in allowed_keys:
              try:
                  if splited_instance[0] == "Date":
                    dict_instance['day_name'] = re.search('Sun|Mon|Tue|Wed|Thu|Fri|Sat',":".join(splited_instance[1:])).group(0)
                    dict_instance[splited_instance[0]] = datetime.strptime(re.search("\d{1,2}\s\w{3}\s\d{4}",":".join(splited_instance[1:])).group(0), '%d %b %Y')
                  elif splited_instance[0] == "From":
                    dict_instance[splited_instance[0]] =  re.search("[\w\-.]+@[\w\-.]+\.\w{2,3}",":".join(splited_instance[1:])).group(0)
                  elif splited_instance[0] == 'X-From':
                    dict_instance[splited_instance[0]] =  re.search('[A-Z][a-z]+([,\s]*[A-Z][a-z]*)*',":".join(splited_instance[1:])).group(0)
                  elif splited_instance[0] in ["To", "Cc", "Bcc"] :
                    dict_instance[splited_instance[0]] = []
                    for dist in ":".join(splited_instance[1:]).split(','):
                      dict_instance[splited_instance[0]].append(re.search("[\w\-.]+@[\w\-.]+\.\w{2,3}",dist).group(0))
                  elif splited_instance[0] in ['X-To', 'X-cc', 'X-bcc'] :
                    dict_instance[splited_instance[0]] = []
                    if '>' in ":".join(splited_instance[1:]) and '<' in ":".join(splited_instance[1:]) and ">@"not in ":".join(splited_instance[1:]) :
                      to_names = ":".join(splited_instance[1:]).split('>, ')
                    else:
                      if "  " in ":".join(splited_instance[1:]):
                        to_names = ":".join(splited_instance[1:]).split('  ')
                      else:
                        to_names = ":".join(splited_instance[1:]).split(', ')

                    for name_dist in to_names:
                      probable_name = re.search('[A-Z][a-z]*([,\s]*[A-Z][a-z]*)*',name_dist)

                      if probable_name:
                        probable_name = probable_name.group(0)
                        if probable_name:
                          dict_instance[splited_instance[0]].append(probable_name)
                      else:
                        probable_name = re.search('[A-Z][a-z]*([,\s]*[A-Z][a-z]*)*',name_dist.title())
                        if probable_name:
                          probable_name = probable_name.group(0)
                          if probable_name:
                            dict_instance[splited_instance[0]].append(probable_name)
                        else:
                          dict_instance[splited_instance[0]].append(re.search('[\w\-.]+@[\w\-.]+\.\w{2,3}',name_dist).group(0))
              except:
                continue
          
          dict_instance.update(
              {
                  'data_content' : content_mail,
                  'path_email' : member.name
              }
          )

          data_simplified.append(dict_instance)

In [ ]:
data_simplified= data_simplified[:2]

In [ ]:
content = {
    'path_email' : [],
    "date": [],
    'day_name' : [],
    "date_id" : [],
    "week" : [],
    "From": [],
    "To": [],
    "X-From" : [],
    "X-To" : [],
}
day_code = {
    'Sun':1,'Mon':2,'Tue':3,'Wed':4,'Thu':5,'Fri':6,'Sat':7
}
begin_date = date.fromisoformat('1998-10-30')
for example in data_simplified:
  
  if 'To' in example:
    for i in range(len(example['To'])):
      content['path_email'].append(example['path_email'])
      content['date'].append(example['Date'])
      content['date_id'].append(example['Date'].date()-begin_date)
      content['day_name'].append(day_code[example['day_name']])
      content['week'].append(math.floor(int((example['Date'].date()-begin_date).days)/7))
      content['From'].append(example['From'])
      if 'X-From' in example:
        content['X-From'].append(" ".join(sorted(example['X-From'].replace(',','').lower().split(' '))))
      else:
        content['X-From'].append("")
      content['To'].append(example['To'][i])
      if i < len(example['X-To']):
        content['X-To'].append(" ".join((sorted(example['X-To'][i].replace(',','').lower().split(' ')))))
      else:
        content['X-To'].append("")
  
  if CC_CONTENT:
    if 'Cc' in example:
      for i in range(len(example['Cc'])):
        content['path_email'].append(example['path_email'])
        content['date'].append(example['Date'])
        content['date_id'].append(example['Date'].date()-begin_date)
        content['day_name'].append(day_code[example['day_name']])
        content['week'].append(math.floor(int((example['Date'].date()-begin_date).days)/7))
        content['From'].append(example['From'])
        if 'X-From' in example:
          content['X-From'].append(" ".join(sorted(example['X-From'].replace(',','').lower().split(' '))))
        else:
          content['X-From'].append("")
        content['To'].append(example['Cc'][i])
        if i < len(example['X-cc']):
          content['X-To'].append(" ".join(sorted(example['X-cc'][i].replace(',','').lower().split(' '))))
        else:
          content['X-To'].append("")
  
  if BCC_CONTENT:
    if 'bcc' in example:
      for i in range(len(example['Bcc'])):
        content['path_email'].append(example['path_email'])
        content['date'].append(example['Date'])
        content['date_id'].append(example['Date'].date()-begin_date)
        content['day_name'].append(day_code[example['day_name']])
        content['week'].append(math.floor(int((example['Date'].date()-begin_date).days)/7))
        content['From'].append(example['From'])
        if 'X-From' in example:
          content['X-From'].append(" ".join(sorted(example['X-From'].replace(',','').lower().split(' '))))
        else:
          content['X-From'].append("")  
        content['To'].append(example['Bcc'][i])
        if i < len(example['X-bcc']):
          content['X-To'].append(" ".join(sorted(example['X-bcc'][i].replace(',','').lower().split(' '))))
        else:
          content['X-To'].append("")


In [ ]:
content = pd.DataFrame.from_dict(content)

In [ ]:
content

,path_email,date,day_name,date_id,week,From,To,X-From,X-To
0,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,patrick.ryder@enron.com,bass eric,patrick ryder
1,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,david.baumbach@enron.com,bass eric,baumbach david
2,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,michael.walters@enron.com,bass eric,michael walters
3,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,jody.crook@enron.com,bass eric,crook jody
4,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,o'neal.winfree@enron.com,bass eric,d o'neal winfree
5,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,denver.plachy@enron.com,bass eric,denver plachy


In [ ]:

content = content[(content['From'].str.contains('@enron.com', regex= True, na=False)) &(content['To'].str.contains('@enron.com', regex= True, na=False))]


In [ ]:
content

,path_email,date,day_name,date_id,week,From,To,X-From,X-To
0,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,patrick.ryder@enron.com,bass eric,patrick ryder
1,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,david.baumbach@enron.com,bass eric,baumbach david
2,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,michael.walters@enron.com,bass eric,michael walters
3,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,jody.crook@enron.com,bass eric,crook jody
4,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,o'neal.winfree@enron.com,bass eric,d o'neal winfree
5,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,denver.plachy@enron.com,bass eric,denver plachy


In [ ]:
content=content[content['From']!=content['To']]

In [ ]:
temp = content[content['X-To'] == '']
temp

,path_email,date,day_name,date_id,week,From,To,X-From,X-To


In [ ]:
for pk_id,content_instance in content.iterrows():
  if str(content_instance['X-To']) == '':
    email = content_instance['To']
    content_to = content[content['To']==email]['X-To']
    content_to_mode = content_to.mode()
    probable_name = content_to_mode
    if str(probable_name[0]):
      content['X-To'][pk_id] = str(probable_name[0])
    else:
      probable_name = content[content['From']==email]['X-From'].mode()
      if len(list(probable_name)) and str(probable_name[0]):
        content['X-To'][pk_id] = str(probable_name[0])
      else:
        content['X-To'][pk_id] = email

In [ ]:
content

,path_email,date,day_name,date_id,week,From,To,X-From,X-To
0,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,patrick.ryder@enron.com,bass eric,patrick ryder
1,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,david.baumbach@enron.com,bass eric,baumbach david
2,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,michael.walters@enron.com,bass eric,michael walters
3,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,jody.crook@enron.com,bass eric,crook jody
4,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,o'neal.winfree@enron.com,bass eric,d o'neal winfree
5,maildir/bass-e/sent/9.,2000-12-06,4,768 days,109,eric.bass@enron.com,denver.plachy@enron.com,bass eric,denver plachy


In [ ]:
set_names = sorted(set(list(content["X-From"]) + list(content["X-To"])))
dict_id_name = {}

for idx,name in enumerate(set_names):
  dict_id_name[name] = idx

content['id-X-From'] = content['X-From']
content['id-X-To'] = content['X-To']
for idx,row in content.iterrows():
  content['id-X-From'][idx] = dict_id_name[content['X-From'][idx]]
  content['id-X-To'][idx] = dict_id_name[content['X-To'][idx]]


<ipython-input-64-572545b7ad39>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['id-X-From'] = content['X-From']
<ipython-input-64-572545b7ad39>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['id-X-To'] = content['X-To']
<ipython-input-64-572545b7ad39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for 

In [ ]:
content

,path_email,date,day_name,date_id,week,From,To,X-From,X-To,id-X-From,id-X-To
0,maildir/blair-l/sent_items/604.,2001-08-02,5,1007 days,143,lynn.blair@enron.com,shelley.corman@enron.com,blair lynn,corman shelley,1080,2193
1,maildir/blair-l/sent_items/318.,2001-10-24,4,1090 days,155,lynn.blair@enron.com,e..anderson@enron.com,blair lynn,anderson e gary,1080,391
2,maildir/blair-l/sent_items/354.,2001-05-31,5,944 days,134,lynn.blair@enron.com,sharon.brown@enron.com,blair lynn,brown sharon,1080,1408
3,maildir/blair-l/sent_items/169.,2001-10-01,2,1067 days,152,lynn.blair@enron.com,britt.davis@enron.com,blair lynn,britt davis,1080,1370
4,maildir/blair-l/sent_items/388.,2001-06-06,4,950 days,135,lynn.blair@enron.com,jerry.wilkens@enron.com,blair lynn,jerry wilkens,1080,4492
...,...,...,...,...,...,...,...,...,...,...,...
187419,maildir/campbell-l/_sent_mail/167.,2000-07-25,3,634 days,90,larry.campbell@enron.com,carmen.chavira@enron.com,campbell f larry,carmen chavira,1576,1628
187420,maildir/campbell-l/_sent_mail/134.,2000-08-16,4,656 days,93,larry.campbell@enron.com,awilli4@enron.com,campbell f larry,awilli,1576,669
187424,maildir/campbell-l/_sent_mail/180.,2001-04-18,4,901 days,128,larry.campbell@enron.com,john.kinser@enron.com,campbell f larry,john,1576,4577
187426,maildir/campbell-l/_sent_mail/162.,2000-07-31,2,640 days,91,larry.campbell@enron.com,matthew.fleming@enron.com,campbell f larry,b fleming matthew,1576,692


In [ ]:
content.to_csv('/content/drive/MyDrive/cleaned_data')

In [ ]:
import pickle
with open('/content/drive/MyDrive/dict_id_name.p', 'wb') as fp:
    pickle.dump(dict_id_name, fp, protocol=pickle.HIGHEST_PROTOCOL)

# ***• Question 1.1.*** Pour chaque collaborateur au sein de l’entreprise, quel est en moyenne le nombre de ses interlocuteurs pendant une semaine de travail?

In [ ]:
content = pd.read_csv('/content/drive/MyDrive/cleaned_data')

In [ ]:
def mean(person_id, dataFrame):
  temp_dataframe = dataFrame[((dataFrame['id-X-From']==person_id) | (dataFrame['id-X-To']==person_id))].groupby(['week'])
  temp_dataframe = pd.DataFrame({'count' : temp_dataframe.size()}).reset_index()
  if len(temp_dataframe)>1:
    return round(temp_dataframe['count'].sum()/ (temp_dataframe.iloc[-1]['week'] - temp_dataframe.iloc[0]['week']), 2)
  else:
    return temp_dataframe['count'].sum()

In [ ]:
set_persons = set(list(content['id-X-From']) + list(content['id-X-To']))
mean_of_interlocutors_foreach_person = {}
for person in set_persons:
    mean_of_interlocutors_foreach_person[person] = mean(person,content)

6576

In [ ]:
mean_of_interlocutors_foreach_person

{0: 1.83,
 1: 1,
 2: 2,
 3: 0.25,
 4: 1,
 5: 0.62,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 0.4,
 12: 1,
 13: 1.45,
 14: 1,
 15: 0.33,
 16: 0.14,
 17: 1,
 18: 2,
 19: 0.31,
 20: 0.35,
 21: 0.18,
 22: 1.23,
 23: 0.14,
 24: 1.66,
 25: 0.75,
 26: 0.91,
 27: 1,
 28: 0.27,
 29: 0.08,
 30: 1,
 31: 0.09,
 32: 0.6,
 33: 2,
 34: 0.06,
 35: 0.07,
 36: 2.0,
 37: 0.1,
 38: 1.67,
 39: 1,
 40: 2,
 41: 0.04,
 42: 0.22,
 43: 0.07,
 44: 4,
 45: 4.0,
 46: 1.0,
 47: 0.51,
 48: 0.47,
 49: 0.66,
 50: 0.29,
 51: 1,
 52: 0.07,
 53: 0.18,
 54: 8.23,
 55: 1.67,
 56: 0.75,
 57: 1,
 58: 16.09,
 59: 9.0,
 60: 3,
 61: 4,
 62: 0.2,
 63: 3.0,
 64: 2.58,
 65: 4.08,
 66: 0.1,
 67: 0.33,
 68: 0.35,
 69: 1,
 70: 0.5,
 71: 2.42,
 72: 0.25,
 73: 3.26,
 74: 1,
 75: 0.27,
 76: 2.04,
 77: 0.27,
 78: 0.67,
 79: 1,
 80: 0.29,
 81: 2,
 82: 0.13,
 83: 1,
 84: 1,
 85: 0.14,
 86: 2.0,
 87: 0.24,
 88: 0.38,
 89: 2,
 90: 1,
 91: 0.17,
 92: 1.0,
 93: 1.01,
 94: 1.0,
 95: 0.26,
 96: 0.18,
 97: 1,
 98: 2,
 99: 1,
 100: 0.98,
 101: 2,
 

In [ ]:
content[content['From']=='vince.kaminski@enron.com']['id-X-From'].unique()

array([4212])

In [ ]:
mean_of_interlocutors_foreach_person[4205]

1.0

In [ ]:
type(max(mean_of_interlocutors_foreach_person.keys()))

int

In [ ]:
import pickle
with open('/content/drive/MyDrive/Q1_1.p', 'wb') as fp:
    pickle.dump(mean_of_interlocutors_foreach_person, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
mean_of_interlocutors_foreach_person

{0: 1.83,
 1: 1,
 2: 2,
 3: 0.25,
 4: 1,
 5: 0.62,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 0.4,
 12: 1,
 13: 1.45,
 14: 1,
 15: 0.33,
 16: 0.14,
 17: 1,
 18: 2,
 19: 0.31,
 20: 0.35,
 21: 0.18,
 22: 1.23,
 23: 0.14,
 24: 1.66,
 25: 0.75,
 26: 0.91,
 27: 1,
 28: 0.27,
 29: 0.08,
 30: 1,
 31: 0.09,
 32: 0.6,
 33: 2,
 34: 0.06,
 35: 0.07,
 36: 2.0,
 37: 0.1,
 38: 1.67,
 39: 1,
 40: 2,
 41: 0.04,
 42: 0.22,
 43: 0.07,
 44: 4,
 45: 4.0,
 46: 1.0,
 47: 0.51,
 48: 0.47,
 49: 0.66,
 50: 0.29,
 51: 1,
 52: 0.07,
 53: 0.18,
 54: 8.23,
 55: 1.67,
 56: 0.75,
 57: 1,
 58: 16.09,
 59: 9.0,
 60: 3,
 61: 4,
 62: 0.2,
 63: 3.0,
 64: 2.58,
 65: 4.08,
 66: 0.1,
 67: 0.33,
 68: 0.35,
 69: 1,
 70: 0.5,
 71: 2.42,
 72: 0.25,
 73: 3.26,
 74: 1,
 75: 0.27,
 76: 2.04,
 77: 0.27,
 78: 0.67,
 79: 1,
 80: 0.29,
 81: 2,
 82: 0.13,
 83: 1,
 84: 1,
 85: 0.14,
 86: 2.0,
 87: 0.24,
 88: 0.38,
 89: 2,
 90: 1,
 91: 0.17,
 92: 1.0,
 93: 1.01,
 94: 1.0,
 95: 0.26,
 96: 0.18,
 97: 1,
 98: 2,
 99: 1,
 100: 0.98,
 101: 2,
 

# ***• Question 1.2.*** Y a-t-il des binômes parmi les collaborateurs d’Enron et si oui lesquels?

In [ ]:
def messages_mean_binome(binome,dataFrame):
  temp_dataframe = dataFrame[(
      ((dataFrame['id-X-From']==binome[0]) & (dataFrame['id-X-To']==binome[1]))
      |
      ((dataFrame['id-X-From']==binome[1]) & (dataFrame['id-X-To']==binome[0]))
    )].groupby(['week'])
  temp_dataframe = pd.DataFrame({'count' : temp_dataframe.size()}).reset_index()
  if len(temp_dataframe)>1:
    return temp_dataframe['count'].sum()/ (temp_dataframe.iloc[-1]['week'] - temp_dataframe.iloc[0]['week'])
  else:
    return temp_dataframe['count'].sum()

In [ ]:
binomes = set(content[['id-X-From','id-X-To']].itertuples(index=False, name=None))
binomes_mean = {}
for binome in binomes:
    if (binome[1],binome[0]) not in binomses_mean:
        binomes_mean[binome] = messages_mean_binome(binome, content)

In [ ]:

my_binomes = {k: v for k, v in binomes_mean.items() if v >5}


{(706, 3815): 9.333333333333334,
 (1527, 2358): 6.0,
 (1635, 3352): 5.428571428571429,
 (4871, 1102): 11.0,
 (4871, 4256): 6.666666666666667,
 (811, 3132): 18.0,
 (2357, 2479): 6.258928571428571,
 (2357, 3647): 5.4,
 (4209, 2512): 6.906976744186046,
 (3947, 1049): 6.0,
 (3343, 2442): 7.0,
 (924, 2098): 6.8,
 (58, 253): 5.4,
 (1904, 1276): 14.0,
 (5329, 5699): 10.0,
 (4009, 6253): 6.0,
 (3830, 3231): 6.0,
 (418, 2611): 6.0,
 (4804, 6446): 7.5,
 (924, 2287): 10.0,
 (5369, 1625): 6.0,
 (4698, 4315): 10.0,
 (924, 690): 8.0,
 (5329, 990): 14.0,
 (924, 1979): 8.0,
 (3554, 6168): 6.0,
 (4840, 2141): 5.25,
 (6394, 4615): 7.1875,
 (1819, 2886): 8.0,
 (4151, 122): 8.0,
 (5491, 6423): 7.0,
 (6533, 1286): 5.333333333333333,
 (460, 6424): 7.0,
 (3554, 2950): 9.0,
 (2920, 1393): 5.2,
 (4871, 3743): 26.0,
 (1797, 4394): 6.0,
 (606, 924): 6.0,
 (2479, 145): 5.611111111111111,
 (4871, 6347): 6.3125,
 (5549, 744): 6.0,
 (4525, 4998): 6.0,
 (1080, 5063): 6.064516129032258,
 (3947, 2025): 5.66666666666666

In [ ]:
with open('/content/drive/MyDrive/Q1_2.p', 'wb') as fp:
    pickle.dump(my_binomes, fp, protocol=pickle.HIGHEST_PROTOCOL)

## ***• Question 1.4. Mêmes questions que 1.2 et 1.3 en remplaçant binôme par trinôme, par groupe de 4, 5, . . .***

In [ ]:
def messages_mean_Nnome(n_nome,dataFrame):
  temp_dataframe = dataFrame[(
      ((dataFrame['id-X-From'].isin(n_nome)) & (dataFrame['id-X-To'].isin(n_nome)))
    )].groupby(['week'])
  temp_dataframe = pd.DataFrame({'count' : temp_dataframe.size()}).reset_index()
  if len(temp_dataframe)>1:
    return temp_dataframe['count'].sum()/ (temp_dataframe.iloc[-1]['week'] - temp_dataframe.iloc[0]['week'])
  else:
    return temp_dataframe['count'].sum()

In [ ]:
trinomes = []
for binome_1 in my_binomes:
  for binome_2 in my_binomes:
    sorted_binome_1 = tuple(sorted(list(binome_1)))
    sorted_binome_2 = tuple(sorted(list(binome_2)))
    if sorted_binome_1 != sorted_binome_2:
      if binome_1[0] in binome_2 :
        trinome = tuple(sorted([binome_1[1],binome_2[0],binome_2[1]]))
        if trinome not in trinomes:
          trinomes.append(trinome)
      if binome_1[1] in binome_2:
        trinome = tuple(sorted([binome_1[0],binome_2[0],binome_2[1]]))
        if trinome not in trinomes:
          trinomes.append(trinome)

In [ ]:
trinomes = set(trinomes)

In [ ]:
n_nomes_mean = {}
for trinome in trinomes:
    n_nomes_mean[trinome] = messages_mean_Nnome(trinome, content)

In [ ]:
my_trinomes = {k: v for k, v in n_nomes_mean.items() if v >5}
my_trinomes

{(1904, 3461, 4662): 5.631578947368421,
 (914, 1631, 4871): 13.753086419753087,
 (1319, 2928, 4840): 7.0,
 (1080, 1228, 1465): 7.580645161290323,
 (2141, 4144, 4840): 7.75,
 (599, 1901, 4998): 14.0,
 (914, 4871, 6309): 6.888888888888889,
 (1102, 4871, 6347): 6.6125,
 (1361, 2781, 6292): 9.75,
 (2141, 2928, 4840): 6.0,
 (1943, 2357, 4222): 13.526315789473685,
 (1819, 2106, 4896): 16.0,
 (1324, 1624, 4871): 9.557692307692308,
 (2721, 4871, 6347): 6.4125,
 (140, 4871, 6309): 11.670886075949367,
 (699, 2239, 4212): 10.54054054054054,
 (1635, 2161, 6340): 8.0,
 (2950, 3554, 6561): 16.0,
 (3324, 4074, 5845): 18.0,
 (144, 2357, 2479): 6.991071428571429,
 (140, 4437, 4871): 11.810126582278482,
 (1705, 2141, 4840): 10.458333333333334,
 (1631, 4871, 6309): 7.125,
 (1705, 4840, 4945): 26.708333333333332,
 (3231, 4144, 4840): 25.083333333333332,
 (2141, 3231, 4840): 20.833333333333332,
 (2025, 2236, 3947): 12.0,
 (2920, 3149, 6315): 6.373134328358209,
 (811, 2215, 3132): 12.0,
 (1899, 2268, 3568):

In [ ]:
with open("/content/drive/MyDrive/Q1_3.p",'wb') as fp:
  pickle.dump(my_trinomes, fp, protocol=pickle.HIGHEST_PROTOCOL)
